# 2단원 연습

In [1]:
flightData2015=spark\
.read\
.option("inferSchema","true")\
.option("header","true")\
.csv("file:///home/ubuntu/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary.csv")

In [2]:
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [3]:
flightData2015.sort("count").explain()

== Physical Plan ==
*(2) Sort [count#12 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#12 ASC NULLS FIRST, 200)
   +- *(1) FileScan csv [DEST_COUNTRY_NAME#10,ORIGIN_COUNTRY_NAME#11,count#12] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/ubuntu/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary...., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>


# 5단원 실습

In [4]:
#데이터 프레임 생성
df=spark.read.format("json").load("file:///home/ubuntu/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json")

In [5]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [6]:
spark.read.format("json").load("file:///home/ubuntu/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json").schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

In [7]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema=StructType([StructField("DEST_COUNTRY_NAME",StringType(),True),
                           StructField("ORIGIN_COUNTRY_NAME",StringType(),True),
                           StructField("count",LongType(),False,metadata={"hello":"world"})])
df=spark.read.format("json").schema(myManualSchema)\
.load("file:///home/ubuntu/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json")

컬럼

In [8]:
from pyspark.sql.functions import col,column
col("someColumnName") #방법1
column("someColumnName") #방법2

Column<b'someColumnName'>

# 질문. 128pg 명시적 컬럼 참조 이거 코드가 안먹음. 뭐임?

5.2.2표현식 이거 뭐지? 이해 잘안됨

In [9]:
from pyspark.sql.functions import expr
expr("(((someCol+5)*200)-6)<otherCol")

Column<b'((((someCol + 5) * 200) - 6) < otherCol)'>

5.3 레코드와 로우

In [10]:
df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

5.4 데이터프레임

In [11]:
#테이터프레임 임시뷰로 만들기
df.createOrReplaceTempView("dfTable")


In [12]:
df.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [13]:
#row들 여러개로 데이터 프레임 만들기
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema=StructType([StructField("some",StringType(),True),
                           StructField("col",StringType(),True),
                           StructField("names",LongType(),False)])
myRow=Row("hello",None,1)
myDf=spark.createDataFrame([myRow],myManualSchema)
myDf.show()

+-----+----+-----+
| some| col|names|
+-----+----+-----+
|hello|null|    1|
+-----+----+-----+



In [14]:
df.select("DEST_COUNTRY_NAME").show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [15]:
#이렇게도 가능함.
df.select(
expr("DEST_COUNTRY_NAME"),
col("DEST_COUNTRY_NAME"),
column("DEST_COUNTRY_NAME"))\
.show(2)

+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+
only showing top 2 rows



In [16]:
#AS를 써서 컬럼명 바꾸기
df.select(expr("DEST_COUNTRY_NAME AS desitnation")).show(2)

+-------------+
|  desitnation|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows



In [17]:
#.alias로 다시 바꿀수 있다.
df.select(expr("DEST_COUNTRY_NAME AS desitnation").alias("DEST_COUNTRY_NAME")).show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



selectExpr 이걸 많으 쓴다고 하는데 이게 솔직히 뭔지는 잘 모르겠음.

In [18]:
df.selectExpr("DEST_COUNTRY_NAME as newColumnName", "DEST_COUNTRY_NAME").show(2)

+-------------+-----------------+
|newColumnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows



아 이게 새로운 데이터 프레임을 생성하게 해주는거구나.

In [19]:
#이거하면 새로운 컬럼이 생긴다.
df.selectExpr("*",
             "(DEST_COUNTRY_NAME=ORIGIN_COUNTRY_NAME) as withinCountry").show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [20]:
#집계를 할 수도 있다.
df.selectExpr("avg(count) AS avgcount","count(distinct(DEST_COUNTRY_NAME))").show(2)

+-----------+---------------------------------+
|   avgcount|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



In [21]:
#명시적인 값을 스파트에게 주기. (리터럴 값 주기)
from pyspark.sql.functions import lit
df.select(expr("*"),lit(1).alias("One")).show(2)
#전체 데이터 프레임을 가져오는데 명시적인 값1 리터럴 형식을  컬럼명 one으로 가져오는거다.

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



In [22]:
#column 추가하기
df.withColumn('numberone',lit(1)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberone|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [23]:
#이렇게도 가능
df.withColumn('withinCountry',expr("ORIGIN_COUNTRY_NAME==DEST_COUNTRY_NAME")).show(2)
#withColumn은 컬럼명, 값을 생성할 표현식이다.

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [24]:
#withcolumn으로 컬럼명 변경 가능, Destination 컬럼을 만들어냈다.
df.withColumn("Destination",expr("DEST_COUNTRY_NAME")).columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count', 'Destination']

In [25]:
#제대로 하면 withColumnRenamed써야함.
df.withColumnRenamed("DEST_COUNTRY_NAME","dest").columns

['dest', 'ORIGIN_COUNTRY_NAME', 'count']

In [26]:
#예약문자는 백틱 문자 사용해서 이스케이핑하기
#selectExpr는 그렇게 하기.

In [27]:
#두개가 똑같다.
df.filter(col("count")<2).show(2) 
df.where("count<2").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [28]:
#여러개 필터링 할때는 그냥 한번에 넣어라.
df.where(col("count")<2).where(col("ORIGIN_COUNTRY_NAME")!="Croatia").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [29]:
#고유한 값 얻으려면 distinct쓰기
#무작위 샘플은 smaple 메서드 사용.

In [30]:
#임의 분할
seed=5
dataFrames=df.randomSplit([0.25,0.75],seed)
dataFrames[0].count()>dataFrames[1].count() #검증용임
dataFrames[0].show(2) #임의 분할중 첫번째 것

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Algeria|      United States|    4|
|          Austria|      United States|   62|
+-----------------+-------------------+-----+
only showing top 2 rows



In [31]:
#union의 경우 동일한 스키마 가져야함. 148페이지 보기

In [32]:
df.sort("count").show(5)
df.orderBy("count","DEST_COUNTRY_NAME").show(5) #두개 조건을 준거임.
df.orderBy(col("count"),col("DEST_COUNTRY_NAME")).show(5) #위랑 같음.

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--

In [33]:
#기준을 명확히 하기위해서 asc, desc사용
df.orderBy(expr("count")).show(2)
df.orderBy(expr("count desc")).show(2)
df.orderBy(expr("count").desc()).show(2)
df.orderBy(col("count").desc()).show(2)
#sortWithPartitions으로 파티선별 정렬을 수행할 수 있다. 처음 spark.read.format할때 

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Moldova|      United States|    1|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Moldova|      United States|    1|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
+-----------------+-------------------+------+
only showing top 2 rows

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| coun

In [34]:
#limit은 로우 수 제한하기
#repartitioning ->자주 필터링하는 컬럼을 기준으로 분할하기
df.rdd.getNumPartitions() #파티션 1개
df=df.repartition(5) #5개로 리파티션
df.rdd.getNumPartitions()

5

In [35]:
df.repartition(col("DEST_COUNTRY_NAME"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

In [36]:
#파티션수 지정 가능
df=df.repartition(4,col("DEST_COUNTRY_NAME"))

In [37]:
df.rdd.getNumPartitions()

4

In [38]:
#coalesce는 전체 데이터 셔플 하지 않고 파티션 병합하는 경우에 사용.

In [39]:
df.repartition(4,col("DEST_COUNTRY_NAME")).coalesce(2) #이렇게 하면 됨.

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

# 6장 실습 코드

In [40]:
df= spark.read.format("csv")\
.option("header","true")\
.option("inferSchema","true")\
.load("file:///home/ubuntu/Spark-The-Definitive-Guide/data/retail-data/by-day/2010-12-01.csv")
df.printSchema()
df.createOrReplaceTempView("dfTable")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [41]:
df.show(10)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

In [42]:
# 스파크 데이터 타입으로 바꾸기

In [43]:
from pyspark.sql.functions import lit
df.select(lit(5),lit("five"),lit(5.0))

DataFrame[5: int, five: string, 5.0: double]

In [44]:
#불리언 데이터 타입 invoiceNum특정 값에 아닌거에 Description들 포함해서 보여주기.
from pyspark.sql.functions import col
df.where(col("InvoiceNo")!=536365)\
.select("InvoiceNo","Description")\
.show(5,False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



In [47]:
#불리언 쓸 때 and는 메서드를 묶어 차례대로 필터를 적용해야한다.
#차례대로 필터 적용할 때는 and는 차례대로, or 구문은 동일한 구문에 조건 정의해야함.
from pyspark.sql.functions import instr
priceFilter=col("UnitPrice")>600
descriptFilter=instr(df.Description,"POSTAGE")>=1
df.where(df.StockCode.isin("DOT")).where(priceFilter|descriptFilter).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [48]:
df.withColumn("isExpensive",expr("NOT UnitPrice<=250"))\
.where("isExpensive")\
.select("Description","UnitPrice").show(5)

+--------------+---------+
|   Description|UnitPrice|
+--------------+---------+
|DOTCOM POSTAGE|   569.77|
|DOTCOM POSTAGE|   607.49|
+--------------+---------+



In [51]:
#수치형 데이터 다루기
from pyspark.sql.functions import expr,pow
fabricatedQuantity=pow(col("Quantity")*col("UnitPrice"),2)+5
df.select(expr("CustomerId"),fabricatedQuantity.alias("realQuantity")).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [54]:
#rount는 소수점 반올림 bround는 내림
from pyspark.sql.functions import lit, round,bround
df.select(round(lit("2.5")),bround(lit("2.5"))).show(2)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 2 rows



In [55]:
#상관관계는 corr ,요약통계는 describe 
df.describe().show()

+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|15.638659854603892|1854.4496996893627|          null|
|    min|           536365|             10002| 4 PURPLE FLOCK D...|               -24|               0.0|           12431.0|     Australia|
|    max|          C

In [56]:
from pyspark.sql.functions import count,mean,stddev_pop,min,max
o1Name="UnitPrice"
quantileProbs=[0.5]
realError=0.05
df.stat.approxQuantile("UnitPrice",quantileProbs,realError)

[2.51]

In [57]:
# 교차표는 corsstab으로 나타낼 수 있다.
df.stat.crosstab("StockCode","Quantity").show()

+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|StockCode_Quantity| -1|-10|-12| -2|-24| -3| -4| -5| -6| -7|  1| 10|100| 11| 12|120|128| 13| 14|144| 15| 16| 17| 18| 19|192|  2| 20|200| 21|216| 22| 23| 24| 25|252| 27| 28|288|  3| 30| 32| 33| 34| 36|384|  4| 40|432| 47| 48|480|  5| 50| 56|  6| 60|600| 64|  7| 70| 72|  8| 80|  9| 96|
+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|             22578|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 

In [58]:
#monotonically_increasing_id는 모든 로우에 고유 ID를 추가한다.

6.5문자열 데이터 다루기

In [59]:
#initcap은 주어진 문자열에서 공백으로 나뉜 모든 단어의 첫글자를 대문자로 변경
#lower, upper로 파이썬처럼 하는거임.
#ltrim,rpad,rtrim는 공백 지우기임.

In [61]:
#정규표현식 기존에 있던 문자 다른걸로 바꾸기.
#치환은 translate를 쓴다.translate(Description,'LEET','1337')
#위 경우 Description 컬럼에서 L->1 E->3 E->3 T->7이렇게 간다.
#regexp_extract는 처음 나온 문자 추출 가능
from pyspark.sql.functions import regexp_replace
regex_string="BLACK|WHITE|RED|GREEN|BLUE"
df.select(
regexp_replace(col("Description"),regex_string,"COLOR").alias("color_clean"),
col("Description")).show(2)

+--------------------+--------------------+
|         color_clean|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
+--------------------+--------------------+
only showing top 2 rows



In [63]:
#coalesce null이 아닌 첫번째 값을 반환함. 즉 모든 컬럼의 값이 null이 아니면 첫번째 컬럼 반환함.
from pyspark.sql.functions import coalesce
df.select(coalesce(col("Description"),col("CustomerId"))).show()

+---------------------------------+
|coalesce(Description, CustomerId)|
+---------------------------------+
|             WHITE HANGING HEA...|
|              WHITE METAL LANTERN|
|             CREAM CUPID HEART...|
|             KNITTED UNION FLA...|
|             RED WOOLLY HOTTIE...|
|             SET 7 BABUSHKA NE...|
|             GLASS STAR FROSTE...|
|             HAND WARMER UNION...|
|             HAND WARMER RED P...|
|             ASSORTED COLOUR B...|
|             POPPY'S PLAYHOUSE...|
|             POPPY'S PLAYHOUSE...|
|             FELTCRAFT PRINCES...|
|             IVORY KNITTED MUG...|
|             BOX OF 6 ASSORTED...|
|             BOX OF VINTAGE JI...|
|             BOX OF VINTAGE AL...|
|             HOME BUILDING BLO...|
|             LOVE BUILDING BLO...|
|             RECIPE BOX WITH M...|
+---------------------------------+
only showing top 20 rows



In [64]:
#ifnull 첫번째 값이 null이면 두번째 값 반환 첫번째 값이 null이 아니면 첫번째거 반환
#nullif는 두 값이 같으면 null
#nv1은 첫번째 값이 null이면 두번째 값 반환
#drop: null가진 로우 제거 all이 조건을 ㅗ들어가면 로우중 하나라도 null이면 지워짐
#df.na.fill(바꿀것) 하면 na값 바꿀것 값으로 바뀜
df.na.drop("all",subset=["StockCode","InvoiceNo"])

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string]

In [65]:
#구조체
from pyspark.sql.functions import struct
complexDF=df.select(struct("Description","InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")
#복합데이터를 가진 데이터 프레임임. 이거 getField를 메서드를 사용해서 조회할 수 있음

In [66]:
#이렇게
complexDF.select(col("complex").getField("Description"))

DataFrame[complex.Description: string]

In [68]:
#배열
from pyspark.sql.functions import split
df.select(split(col("Description")," ")).show(2)

+---------------------+
|split(Description,  )|
+---------------------+
| [WHITE, HANGING, ...|
| [WHITE, METAL, LA...|
+---------------------+
only showing top 2 rows



In [69]:
df.select(split(col("Description")," ").alias("array_col"))\
.selectExpr("array_col[0]").show(2)

+------------+
|array_col[0]|
+------------+
|       WHITE|
|       WHITE|
+------------+
only showing top 2 rows



In [72]:
#size크기 확인,array_contain으로 배열에 특정 값 있는지 확인
#explode 지정된 값만 배열로 입력된 컬럼을 모두 로우로 변환함. 나머지는 중복해서 표시함 188페이지 표시 보기
#map은 함수가 컬럼의 키-값쌍을 이용해 생성
from pyspark.sql.functions import create_map
df.select(create_map(col("Description"),col("InvoiceNo")).alias("complex_map"))\
.show(2)

+--------------------+
|         complex_map|
+--------------------+
|[WHITE HANGING HE...|
|[WHITE METAL LANT...|
+--------------------+
only showing top 2 rows



In [ ]:
#json도 다룰 수 있음.

In [ ]:
#사용자 정의 함수는 그냥 우리가 def만들어서 하면 된다. UDF라고 명명함.